# NumPy arrays and C++ binary IO

cf. Peter Gottschling.  **Discovering Modern C++: An Intensive Course for Scientists, Engineers, and Programmers**, A.2.7 Binary I/O.  

cf. [Writing binary in c++ and read in python](https://stackoverflow.com/questions/37503346/writing-binary-in-c-and-read-in-python)

In [10]:
import numpy
import numpy as np

In [2]:
# find out where we are in the file directory
import os, sys

In [3]:
print(os.getcwd())
datafilefolder = "./data/"

/home/mobicfd/ReacCFD/CompPhys/Cpp/Cpp14/FileIO


### [`numpy.ndarray.tofile`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.tofile.html)  

`ndarray.tofile(fid,sep="",format="%s")`  

Write array to a file as text or binary (default).  

Data always written in 'C' order, independent of order of *a*.  

#### Parameters  
**fid** : *file or str*  
        An open file object, or string containing filename.  
**sep** : *str*
        Separator between array items for text output.  If "" (empty), a binary file is written, equivalent to `file.write(a.tobytes())`  
**format** : *str*
        Format string for text file output.  Each entry in the array is formatted to text by first converting it to closest Python type, and then using "format" % item.  

In [4]:
m=5
n=4

In [5]:
A = 11.111111*np.array(range(m*n),dtype=np.float32).reshape((m,n))

In [6]:
print(A)

[[   0.           11.11111069   22.22222137   33.33333206]
 [  44.44444275   55.55555344   66.66666412   77.777771  ]
 [  88.8888855   100.          111.11110687  122.22221375]
 [ 133.33332825  144.44444275  155.55554199  166.66665649]
 [ 177.777771    188.8888855   200.          211.11109924]]


In [13]:
Afilename = "A_mat_5_4.npy"

In [9]:
try:
    A.tofile(datafilefolder+ Afilename )
except IOError:
    if not os.path.exists(datafilefolder):
        os.makedirs(datafilefolder)

In [17]:
print(os.listdir(datafilefolder))
print(os.listdir(os.getcwd()))

['A_mat_5_4.npy']
['data', '.ipynb_checkpoints', 'FileIO.ipynb', 'binIO_playground.cu']


### [`numpy.fromfile`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.fromfile.html)  
```  
numpy.fromfile(file, dtype=float, count=-1, sep='')  
```  

In [14]:
A_in = np.fromfile(datafilefolder+ Afilename, dtype=np.float32)

In [15]:
print(A_in)

[   0.           11.11111069   22.22222137   33.33333206   44.44444275
   55.55555344   66.66666412   77.777771     88.8888855   100.
  111.11110687  122.22221375  133.33332825  144.44444275  155.55554199
  166.66665649  177.777771    188.8888855   200.          211.11109924]


Then go to CUDA C++14 file `binIO_playground.cu` or the C++14 version (serial version), `binIO_playground.cpp`.  Load it with [`std::ifstream`](http://www.cplusplus.com/reference/fstream/ifstream/), Input stream class to operate on files.  

### `std::ifstream`   
```  
typedef basic_ifstream<char> ifstream;  
```  

### [`reinterpret_cast`](http://en.cppreference.com/w/cpp/language/reinterpret_cast)  
```  
reinterpret_cast < new_type > ( expression )  
```  
Returns a value of type new_type.  

### std::istream_iterator  

Note, cf. https://stackoverflow.com/questions/37588569/using-stdistream-iterator-to-read-binary-data-from-file-stops-prematuraly
" 	`istream_iterator` is an avatar of operator `>>`; it uses that operator to read from the stream. That is almost never what you want for reading binary data, because >> is a formatted input function. You could probably coerce it to do what you want by using manipulators such as noskipws on the stream, but it would still effectively remain a use of the wrong tool for the job.

If you want an iterator-based access to binary data in a stream, you might be better off using an `istreambuf_iterator` (which is guaranteed to work character by character) instead."  Angew

`std::istreambuf_iterator` is a single-pass input iterator that reads successive characters from the `std::basic_streambuf` object for which it was constructed.  